# **Modelling and Evaluation: Regression**

## Objectives

1. Fit and evaluate a regression model which will predict house prices.
2. Answers business criteria 2. Predict prices for client house data.
3. We have agreed an R2 score of at least 0.75 on the train set as well as on the test set.
4. Can predict house prices based on input data from streamlit dashboard. 

## Inputs

1. House_prices_records_clean.csv
2. Inherited_houses_clean.csv
3. Findings of feature engineering notebook

## Outputs

1. Data sets for train, validate and test sets.
2. Feature engineering pipeline.
3. Trained Model.
4. Predictions for client house prices and supporting data.



---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'/workspace/Heritage-Housing/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/workspace/Heritage-Housing'

# Load data

Note that we are using the cleaned data sets from the clean data notebook.

In [4]:
import numpy as np
import pandas as pd

house_prices_clean_df = pd.read_csv(f"outputs/datasets/clean_data/House_prices_records_clean.csv")
inherited_houses_clean_df = pd.read_csv(f"outputs/datasets/clean_data/Inherited_houses_clean.csv")

In [5]:
house_prices_clean_df.head()

,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,GarageArea,GarageFinish,GarageYrBlt,...,LotArea,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,YearBuilt,YearRemodAdd,SalePrice
0,856,854.0,3.0,No,706,GLQ,150,548,RFn,2003.0,...,8450,65.0,196.0,61,5,7,856,2003,2003,208500
1,1262,0.0,3.0,Gd,978,ALQ,284,460,RFn,1976.0,...,9600,80.0,0.0,0,8,6,1262,1976,1976,181500
2,920,866.0,3.0,Mn,486,GLQ,434,608,RFn,2001.0,...,11250,68.0,162.0,42,5,7,920,2001,2002,223500
3,961,0.0,3.0,No,216,ALQ,540,642,Unf,1998.0,...,9550,60.0,0.0,35,5,7,756,1915,1970,140000
4,1145,0.0,4.0,Av,655,GLQ,490,836,RFn,2000.0,...,14260,84.0,350.0,84,5,8,1145,2000,2000,250000


In [6]:
inherited_houses_clean_df.head()

,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,GarageArea,GarageFinish,GarageYrBlt,...,KitchenQual,LotArea,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,YearBuilt,YearRemodAdd
0,896,0,2,No,468.0,Rec,270.0,730.0,Unf,1961.0,...,TA,11622,80.0,0.0,0,6,5,882.0,1961,1961
1,1329,0,3,No,923.0,ALQ,406.0,312.0,Unf,1958.0,...,Gd,14267,81.0,108.0,36,6,6,1329.0,1958,1958
2,928,701,3,No,791.0,GLQ,137.0,482.0,Fin,1997.0,...,TA,13830,74.0,0.0,34,5,5,928.0,1997,1998
3,926,678,3,No,602.0,GLQ,324.0,470.0,Fin,1998.0,...,Gd,9978,78.0,20.0,36,6,6,926.0,1998,1998


---

# Split data

We need to split our house price data into train, validate and test sets. We will follow the conventional ration of 7:1:2,

First we split into train and test sets.

In [11]:
from sklearn.model_selection import train_test_split

TrainSet, TestSet, = train_test_split(house_prices_clean_df,
                                        test_size=0.2,
                                        random_state=0)

print(f"TrainSet shape: {TrainSet.shape} \nTestSet shape: {TestSet.shape}")

TrainSet shape: (1168, 22) 
TestSet shape: (292, 22)


Then we split the test set again. We set the ratio to 0.12 as we want an overall proportion of 7:1:2.

In [12]:
TrainSet, ValidateSet = train_test_split(TrainSet,
                                        test_size=0.12,
                                        random_state=0)

print(f"TrainSet shape: {TrainSet.shape} \n ValidateSet shape: {ValidateSet.shape}")

TrainSet shape: (1027, 22) 
 ValidateSet shape: (141, 22)


---

# Pipeline

We apply the feature engineering steps outlined in the notebook

In [16]:
from sklearn.pipeline import Pipeline
from feature_engine.selection import DropFeatures
from feature_engine.encoding import OrdinalEncoder
from feature_engine import transformation as vt
from feature_engine.outliers import Winsorizer


pipeline = Pipeline([
    ('drop_features', DropFeatures(features_to_drop = ['1stFlrSF', 'GarageArea', 'GarageYrBlt', 'GrLivArea', 'YearRemodAdd']) ),
    ("OrdinalCategoricalEncoder",OrdinalEncoder(encoding_method='arbitrary', 
                                                  variables = ['BsmtExposure',
                                                               'BsmtFinType1',
                                                               'GarageFinish',
                                                               'KitchenQual'] ) ),

    ('pt', vt.PowerTransformer(variables = [ '2ndFlrSF',
                                              'BedroomAbvGr',
                                              'BsmtFinSF1',
                                              'BsmtUnfSF',
                                              'LotArea', 
                                              'LotFrontage', 
                                              'MasVnrArea', 
                                              'OpenPorchSF', 
                                              'OverallCond', 
                                              'OverallQual', 
                                              'TotalBsmtSF', 
                                              'YearBuilt', ]) ),
    ('winsorizer_iqr', Winsorizer(capping_method='iqr', fold=1.5, tail='both'))
  ])

We can then fit our train, validate and test sets with the pipeline.

In [17]:
house_prices_train_set = pipeline.fit_transform(TrainSet)
house_prices_train_set.head()

,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,GarageFinish,KitchenQual,LotArea,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,YearBuilt,SalePrice
759,34.684290,2.000000,1.0,7.483315,0.0,34.612137,0.0,0.0,110.711336,8.062258,22.649503,9.899495,2.236068,2.828427,36.304270,44.665423,290000.0
172,26.514147,1.414214,1.0,22.248595,1.0,18.814888,0.0,0.0,72.842295,6.633250,0.000000,5.916080,2.645751,2.645751,32.619013,44.575778,239000.0
281,0.000000,1.732051,1.0,30.083218,1.0,18.894444,1.0,0.0,84.852814,7.745967,8.246211,10.954451,2.236068,2.449490,35.524639,44.788391,185000.0
565,28.390139,2.000000,1.0,0.000000,2.0,28.390139,2.0,1.0,82.813042,8.124038,0.000000,8.124038,2.000000,2.449490,28.390139,43.760713,128000.0
1236,29.359837,1.414214,1.0,0.000000,2.0,27.640550,0.0,0.0,55.826459,6.032625,10.295630,5.196152,2.236068,2.645751,27.640550,44.754888,175500.0


In [18]:
house_prices_validate_set = pipeline.fit_transform(ValidateSet)
house_prices_validate_set.head()

,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,GarageFinish,KitchenQual,LotArea,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,YearBuilt,SalePrice
1425,0.000000,1.732051,0.0,0.000000,0.0,35.383612,0.0,0.0,103.542262,8.944272,15.588457,6.244998,2.449490,2.449490,35.383612,44.260592,142000.0
383,0.000000,1.732051,0.0,0.000000,0.0,28.000000,0.0,1.0,94.868330,7.745967,0.000000,0.000000,1.915935,2.449490,28.000000,43.908997,76000.0
1287,0.000000,1.732051,1.0,28.495614,1.0,28.495614,0.0,1.0,134.142765,8.306624,24.919872,14.071247,2.236068,2.236068,40.298883,44.317040,190000.0
255,35.860842,1.732051,0.0,0.000000,0.0,31.224990,1.0,0.0,93.477270,8.124038,17.378147,0.000000,2.236068,2.645751,31.224990,44.710178,230000.0
62,0.000000,1.732051,2.0,4.898979,2.0,36.687873,2.0,0.0,80.262071,6.633250,13.341664,7.000000,2.236068,2.828427,37.013511,44.788391,202500.0


In [19]:
house_prices_test_set = pipeline.fit_transform(TestSet)
house_prices_test_set.head()

,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,GarageFinish,KitchenQual,LotArea,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,YearBuilt,SalePrice
529,0.000000,2.000000,0.0,34.914181,0.0,28.565714,0.0,0.0,139.197513,8.306624,0.000000,0.000000,1.915935,2.449490,45.110974,44.237993,200624.0
491,24.899799,1.732051,0.0,20.074860,1.0,15.427249,1.0,1.0,97.416631,8.888194,0.000000,0.000000,2.645751,2.449490,28.390139,44.056782,133000.0
459,14.966630,1.732051,0.0,13.601471,2.0,22.891046,1.0,2.0,83.755597,8.306624,12.688578,0.000000,2.000000,2.236068,26.627054,44.158804,110000.0
279,29.427878,2.000000,0.0,19.798990,1.0,27.712813,2.0,0.0,100.024997,9.110434,17.291616,10.816654,2.236068,2.645751,34.058773,44.463468,192000.0
655,0.000000,1.732051,0.0,0.000000,3.0,22.912878,1.0,0.0,54.527278,6.117275,19.519221,0.000000,2.236068,2.449490,22.912878,44.395946,88000.0


---

# The Model

---

# Predict house prices

# Push files to Repo

* If you do not need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
  # create here your folder
  # os.makedirs(name='')
except Exception as e:
  print(e)
